In [14]:
from gurobipy import Model, GRB

In [15]:
# Create a new model
m = Model("mckenna_logistics")

In [16]:
# Decision variables
# x[i, j] for heavy, y[i, j] for medium, z[i, j] for light, where i is the pallet number and j is the truck number
x = m.addVars(7, 9, vtype=GRB.BINARY, name='x')  # Heavy pallets
y = m.addVars(6, 9, vtype=GRB.BINARY, name='y')  # Medium pallets
z = m.addVars(5, 9, vtype=GRB.BINARY, name='z')  # Light pallets
v = m.addVars(9, vtype=GRB.BINARY, name='v')     # Truck chosen

In [17]:
# Costs per trip for each truck type
costs = [5500, 4700, 3900]

In [18]:
# Weight limits for each truck type
weight_limits = [13.5, 12, 10]

In [19]:
# Pallet weights
pallet_weights = [4, 3, 0.5]  # Heavy, Medium, Light

In [20]:
# Objective Function: Minimize the total cost of shipping
m.setObjective(
    sum(costs[j//3] * v[j] for j in range(9)),
    GRB.MINIMIZE)

In [21]:
# Constraints

# Each pallet is assigned to exactly one truck
for i in range(7):
    m.addConstr(sum(x[i, j] for j in range(9)) == 1, f"HeavyPalletAssign_{i}")

for i in range(6):
    m.addConstr(sum(y[i, j] for j in range(9)) == 1, f"MediumPalletAssign_{i}")

for i in range(5):
    m.addConstr(sum(z[i, j] for j in range(9)) == 1, f"LightPalletAssign_{i}")

# Truck weight limit constraint for each truck
for j in range(9):
    m.addConstr(
        sum(4 * x[i, j] for i in range(7)) +
        sum(3 * y[i, j] for i in range(6)) +
        sum(0.5 * z[i, j] for i in range(5)) <= weight_limits[j//3] * v[j],
        f"WeightLimit_{j}")

# Truck utilization constraint
# Ensure that if any pallet is assigned to a truck, that truck is marked as chosen
for j in range(9):
    m.addConstr(
        sum(x[i, j] for i in range(7)) +
        sum(y[i, j] for i in range(6)) +
        sum(z[i, j] for i in range(5)) <= 18 * v[j],
        f"TruckUse_{j}")

# Each truck type can be used at most three times
for t in range(3):
    m.addConstr(
        sum(v[3*t + k] for k in range(3)) <= 3,
        f"TruckTypeUse_{t}")

In [22]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 39 rows, 171 columns and 513 nonzeros
Model fingerprint: 0x03edde6b
Variable types: 0 continuous, 171 integer (171 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+01]
  Objective range  [4e+03, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 23500.000000
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 36 rows, 171 columns, 504 nonzeros
Variable types: 0 continuous, 171 integer (171 binary)

Root relaxation: objective 1.894583e+04, 47 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 18945.8333    0    9 23500.0000 18945.8333  19.4% 

In [23]:
# Print the solution
if m.status == GRB.OPTIMAL:
    for var in m.getVars():
        if var.x > 0:
            print(f"{var.varName} = {var.x}")

else:
    print('No solution found')

x[0,4] = 1.0
x[1,5] = 1.0
x[2,5] = 1.0
x[3,5] = 1.0
x[4,4] = 1.0
x[5,0] = 1.0
x[6,3] = 1.0
y[0,0] = 1.0
y[1,0] = 1.0
y[2,0] = 1.0
y[3,3] = 1.0
y[4,3] = 1.0
y[5,4] = 1.0
z[0,0] = 1.0
z[1,3] = 1.0
z[2,3] = 1.0
z[3,3] = 1.0
z[4,3] = 1.0
v[0] = 1.0
v[3] = 1.0
v[4] = 1.0
v[5] = 1.0
